# 챗봇 만들기(Char+CNN / Bi-LSTM)

In [ ]:
train_data_order = ['2017년식 쏘나타에 주행중 소음이나서 p0300 발생 브레이크 패드 교체하고 시운전 후 이상없음']
train_data_reserve = ['정비소 추천 해줄레']
train_data_info = ['2017년식 쏘나타에 주행중 소음이나서 p0300 발생 브레이크 패드 교체하고 시운전 후 이상없음']
bb = ['2017년식 쏘나타에 무슨 문제가 있는지 알려줘']
aa = ['언제 엔진오일 갈았는지 알려줘']


get_data_list = train_data_info[0]

dict_entity = {
    'vehl' : ['쏘나타','아반떼','스팅어','제네시스','산타페','투싼'],
    'year' : ['2017','2016','2015','2014'],
    'psit' : ['소음','누수','시동꺼짐'],
    'part' : ['도어트림','엔진','미션','브레이크','브레이크 패드'],
    'un_repair' : ['재현 안됨','나중에 동일현상 발생 시 재입고 요망' ],
    'condition' : ['정차시','주행','시동시'],
    'center' : ['정비소','정비센터','블루링크','오토큐'],
    'dtc' : ['0300','b0400','c0500'],
    'position' : ['앞','뒤']
}

In [1]:
input_data = "2017년식 쏘나타에 무슨 문제가 있는지 알려줘" #채팅에서 입력
output_data = ""
#챗봇에서 서버와 주고받을 REST format 
request = {
            "intent_id" : "",
            "input_data":input_data, 
            "request_type" : "text",
            "story_slot_entity" : {},
            "output_data" : output_data
          }


# 기본 Data Set (DB)

In [2]:
voc_list = []

intent_list = {
    "정비진단" : ["문제","현상","원인","조치","해결","조건","차종","연식"],
    "정비추천" : ["언제","교환"],
    "정보" : ["정보", "알려", "읽어줘"]
}

ner_list = []

story_slot_entity = {"정비진단": {"현상" : None, "조건" : None, "차종" : None},
                     "정비추천": {"언제" : None, "교환" : None},
                     "정보": {"대상" : None}                    
                    }
menu_list = ['소음','누수','시동꺼짐']

# 형태소 분석

In [3]:
#형태소 분석 mecab사용
from konlpy.tag import Mecab
mecab = Mecab('/usr/local/lib/mecab/dic/mecab-ko-dic') #고유명사는 별도 NNP 등록
preprocessed = mecab.pos(request.get('input_data'))

In [4]:
print (preprocessed)

[('2017', 'SN'), ('년', 'NNBC'), ('식', 'NNB'), ('쏘나타', 'NNP'), ('에', 'JKB'), ('무슨', 'MM'), ('문제', 'NNG'), ('가', 'JKS'), ('있', 'VA'), ('는지', 'EC'), ('알려줘', 'VV+EC+VX+EC')]


# Intent 도출

In [5]:
intent_id ="정비진단"
slot_value = story_slot_entity.get("정비진단")

# NER 도출

In [7]:
B_vhcl = '쏘나타'
B_year = '2017'
B_psit = '소음'
B_dtc='0300'

In [ ]:
    'vehl' : ['쏘나타','아반떼','스팅어','제네시스','산타페','투싼'],
    'year' : ['2017','2016','2015','2014'],
    'psit' : ['소음','누수','시동꺼짐'],
    'part' : ['도어트림','엔진','미션','브레이크','브레이크 패드'],
    'un_repair' : ['재현 안됨','나중에 동일현상 발생 시 재입고 요망' ],
    'condition' : ['정차시','주행','시동시'],
    'center' : ['정비소','정비센터','블루링크','오토큐'],
    'dtc' : ['0300','b0400','c0500']

# Dictionary 기반 Slot 구성

In [12]:
for pos_tag in preprocessed:
    if (pos_tag[1] in ['SN','NNG','MAG', 'NNP','SL']): #명사, 영어만 사용
        if pos_tag[0] in menu_list:
            slot_value["현상"] = pos_tag[0]

In [13]:
print (story_slot_entity.get('정비진단'))

{'현상': None, '차종': None, '조건': None}


# 빈 Slot 검색

In [19]:

if(None in slot_value.values()): #빈 Slot 출력
    key_values = ""
    for key in slot_value.keys():
        if(slot_value[key] is None):
            key_values = key_values + key + ","
    output_data = key_values + '입력해주세요'
            
print (output_data)

현상,차종,조건,입력해주세요


In [22]:
response = {
            "intent_id" : "",
            "input_data":"쏘나타 소음", 
            "request_type" : "text",
            "story_slot_entity" : {},
            "output_data" : ""
          }
response["output_data"]= output_data

In [23]:
print (response['output_data'])

현상,차종,조건,입력해주세요
